# Offer

In [4]:
import pandas as pd
import re
import numpy as np

def preprocess_offer(file_path):
    # 엑셀 파일을 DataFrame으로 불러옵니다.
    offer = pd.read_excel(file_path)

    offer.drop(columns=['Unnamed: 0'], inplace=True)
    offer.drop(offer.index[1157], axis=0, inplace=True)
    offer['Capacity'] = offer['Capacity'].astype(int)
    offer['Min Per Session'] = offer['Min Per Session'].astype(int)

    offer['Session'] = offer['Session'].str.upper()
    offer['Lecturer'] = offer['Lecturer'].str.upper()
    offer['CourseCode'] = offer['CourseCode'].str.upper()
    offer['FacultyCode'] = offer['FacultyCode'].str.upper()

    offer['Session'] = offer['Session'].str.replace(r'\(S\)', '', regex=True)

    # "COMBINED TO /"를 복사하여 error 테이블에 저장. 이후 원본 데이터에서 삭제.
    mask = offer['Session'].str.contains("COMBINED TO /", na=False)

    # error 데이터프레임에 해당 행들을 복사하여 저장합니다.
    error = offer[mask].copy()

    # 원본 offer 데이터프레임에서 해당 행들을 삭제합니다.
    offer.drop(offer[mask].index, inplace=True)

    #"GROUP A", "GROUP B", "GROUP C" 등 해당하는 행을 찾고, Session 값을 "LECTURE"로 변경
    offer.loc[offer['Session'].str.contains(r'^GROUP\s+[A-Z]$', na=False), 'Session'] = 'LECTURE'

    group_cols = ['CourseCode', 'FacultyCode', 'Session', 'Capacity', 'Min Per Session', 'Lecturer']

    # 그룹별로 중복된 행을 처리합니다.
    for _, group in offer.groupby(group_cols):
        if len(group) > 1:  # 중복 그룹인 경우
            # 원래 Session 값을 앞뒤 공백 제거 후 사용합니다.
            orig_session = group.iloc[0]['Session'].strip()
            # Session 값이 숫자로 끝나는지 검사 (예: "TUTORIAL 1")
            m = re.search(r'^(.*?)(\d+)$', orig_session)
            if m:
                base = m.group(1).strip()  # 문자 부분 (예: "TUTORIAL")
                start_num = int(m.group(2))  # 기존에 붙은 숫자 (예: 1)
            else:
                base = orig_session
                start_num = 1

            # 그룹 내 각 행에 대해 순차적으로 번호를 붙여 Session 값을 변경합니다.
            for offset, idx in enumerate(group.index):
                new_session = f"{base} {start_num + offset}"
                offer.at[idx, 'Session'] = new_session

    # (초기 전제: offer DataFrame과 error DataFrame이 존재함; error가 없으면 빈 DataFrame 생성)
    if 'error' not in globals():
        error = pd.DataFrame()

    ### Step 1. offer에서 "COMBINED TO" 행 추출 → child_class로 저장, offer에서는 제거
    child_class = offer[offer['Session'].str.contains('COMBINED TO', na=False)].copy()
    offer.drop(child_class.index, inplace=True)

    ### Step 2. child_class의 Session 열에서 과목코드 추출
    child_class['extracted_code'] = child_class['Session'].str.extract(r'COMBINED TO\s+([A-Z0-9]+)', expand=False)

    ### Step 3. child_class 내에서 오류 및 정상/후손 분리

    # (A) "자기 자신 참조" 조건: 추출한 코드가 해당 행의 CourseCode와 같으면 → 오류
    self_ref = child_class['extracted_code'] == child_class['CourseCode']

    # (B) 정상(child) 조건: 자기 자신 참조가 아니고, 추출한 코드가 offer의 CourseCode에 존재하는 경우
    valid_child_mask = (~self_ref) & (child_class['extracted_code'].isin(offer['CourseCode']))

    # (C) 후손(grandchild) 후보: 자기 자신 참조가 아니고, 추출한 코드가 offer에는 없지만, child_class의 CourseCode에는 존재하는 경우  
    #     (즉, 부모(대상 CourseCode)가 offer에 없으므로 자식끼리 연결되어 있음을 의미)
    valid_grand_mask = (~self_ref) & (~child_class['extracted_code'].isin(offer['CourseCode'])) & \
                        (child_class['extracted_code'].isin(child_class['CourseCode']))

    # (D) 오류 조건:  
    #     - 자기 자신 참조인 경우  
    #     - 또는 추출한 코드가 offer에도 child_class에도 존재하지 않는 경우
    error_mask = self_ref | ((~child_class['extracted_code'].isin(offer['CourseCode'])) & 
                               (~child_class['extracted_code'].isin(child_class['CourseCode'])))

    # child_class 오류 처리: 추출
    child_error = child_class[error_mask].copy()

    # 정상 child_class (valid_child) 추출
    valid_child = child_class[valid_child_mask].copy()

    # 후보 후손(grandchild) 추출
    candidate_grand = child_class[valid_grand_mask].copy()

    ### Step 4. grand_child_class 내에서 추가 오류 처리
    # 오류 조건 (grandchild): 자기 자신 참조 → 오류
    grand_self_ref = candidate_grand['extracted_code'] == candidate_grand['CourseCode']
    error_grand = candidate_grand[grand_self_ref].copy()

    # 최종 grand_child_class: 후보에서 자기 자신 참조 오류 제거
    grand_child_class = candidate_grand[~grand_self_ref].copy()

    # 최종 error: 기존 child 오류와 후손에서 발생한 오류를 모두 누적
    error_df = pd.concat([child_error, error_grand], ignore_index=True)
    error = pd.concat([error, error_df], ignore_index=True)

    ### 최종 정상 데이터
    # child_class는 valid_child로 업데이트 (offer의 부모가 존재하는 경우)
    child_class = valid_child.copy()
    # grand_child_class는 위에서 구한 대로

    # error DataFrame이 없으면 빈 DataFrame 생성
    if 'error' not in globals():
        error = pd.DataFrame()

    ##############################################
    # 교차검증 1: child_class에서 자기 자신 참조하는 행 처리
    ##############################################
    child_self_ref_mask = (child_class['extracted_code'] == child_class['CourseCode'])
    child_self_ref_errors = child_class[child_self_ref_mask].copy()
    error = pd.concat([error, child_self_ref_errors], ignore_index=True)
    child_class = child_class[~child_self_ref_mask].copy()

    ##############################################
    # 교차검증 2: child_class에서 추출한 과목코드가 offer의 CourseCode에 없음(부모 없음)
    ##############################################
    child_missing_parent_mask = ~child_class['extracted_code'].isin(offer['CourseCode'])
    child_missing_parent_errors = child_class[child_missing_parent_mask].copy()
    error = pd.concat([error, child_missing_parent_errors], ignore_index=True)
    child_class = child_class[~child_missing_parent_mask].copy()

    ##############################################
    # 교차검증 3: offer에서 자기 자신 참조하는 행 처리
    # (offer에 COMBINED TO가 남아있다면 해당 행에 대해 extracted_code를 추출)
    ##############################################
    offer_combined_mask = offer['Session'].str.contains('COMBINED TO', na=False)
    if offer_combined_mask.any():
        offer.loc[offer_combined_mask, 'extracted_code'] = offer.loc[offer_combined_mask, 'Session'].str.extract(r'COMBINED TO\s+([A-Z0-9]+)', expand=False)
        offer_self_ref_mask = (offer['extracted_code'] == offer['CourseCode'])
        offer_self_ref_errors = offer[offer_self_ref_mask].copy()
        error = pd.concat([error, offer_self_ref_errors], ignore_index=True)
        offer.drop(offer[offer_self_ref_mask].index, inplace=True)

    ##############################################
    # 교차검증 4: grand_child_class에서 자기 자신 참조하는 행 처리
    ##############################################
    grand_self_ref_mask = (grand_child_class['extracted_code'] == grand_child_class['CourseCode'])
    grand_self_ref_errors = grand_child_class[grand_self_ref_mask].copy()
    error = pd.concat([error, grand_self_ref_errors], ignore_index=True)
    grand_child_class = grand_child_class[~grand_self_ref_mask].copy()

    ##############################################
    # 교차검증 5: grand_child_class에서 추출한 과목코드가 offer의 CourseCode에 없음(부모 없음)
    ##############################################
    grand_missing_parent_mask = ~grand_child_class['extracted_code'].isin(offer['CourseCode'])
    grand_missing_parent_errors = grand_child_class[grand_missing_parent_mask].copy()
    error = pd.concat([error, grand_missing_parent_errors], ignore_index=True)
    grand_child_class = grand_child_class[~grand_missing_parent_mask].copy()

    ##############################################
    # 교차검증 6: 부모(offer 또는 child_class)에서 드랍된 행과 연결된 자식(또는 손자) 행 처리
    # 드랍된 행들의 CourseCode를 모아서, 이 코드를 extracted_code로 사용하는 행이 있다면 error로 처리
    ##############################################
    # 지금까지 error에 추가된 행들의 CourseCode를 모읍니다.
    dropped_codes = set(error['CourseCode'].unique())

    # child_class에서 연결된 자식 행 검사
    child_linked_mask = child_class['extracted_code'].isin(dropped_codes)
    child_linked_errors = child_class[child_linked_mask].copy()
    error = pd.concat([error, child_linked_errors], ignore_index=True)
    child_class = child_class[~child_linked_mask].copy()

    # grand_child_class에서 연결된 손자 행 검사
    grand_linked_mask = grand_child_class['extracted_code'].isin(dropped_codes)
    grand_linked_errors = grand_child_class[grand_linked_mask].copy()
    error = pd.concat([error, grand_linked_errors], ignore_index=True)
    grand_child_class = grand_child_class[~grand_linked_mask].copy()

    # child_class의 각 행을 순회하며 조건에 맞는 offer 행의 Capacity에 child_class의 Capacity를 합산합니다.
    for idx, child_row in child_class.iterrows():
        # 조건: offer의 CourseCode, Min Per Session, Lecturer가 child_row의 extracted_code, Min Per Session, Lecturer와 일치
        mask = (
            (offer['CourseCode'] == child_row['extracted_code']) &
            (offer['Min Per Session'] == child_row['Min Per Session']) &
            (offer['Lecturer'] == child_row['Lecturer'])
        )
        # 일치하는 행이 있다면, offer의 Capacity에 child_row의 Capacity를 더함
        if mask.any():
            offer.loc[mask, 'Capacity'] += child_row['Capacity']

    # 기존 조건
    conditions = [
        offer['Session'].str.contains('LECTURE & TUTORIAL', na=False, case=False),
        offer['Session'].str.contains('TUTORIAL', na=False, case=False),
        offer['Session'].str.contains('LECTURE', na=False, case=False),
        offer['Session'].str.contains('LAB', na=False, case=False),
        offer['Session'].str.contains('PBL 1', na=False, case=False),
        offer['Session'].str.contains('HAND DRAWING|CAD DRAWING 2|CAD DRAWING 1', na=False, case=False),
        offer['Session'].str.contains('KITCHEN', na=False, case=False),
        offer['Session'].str.contains('OPTOM CLINIC|SOO - EXAM CLINIC', na=False, case=False)
    ]

    # 기존 선택지 + 새 선택지
    choices = [
        'GENERAL',
        'TUTORIAL',
        'LECTURE',
        'LAB',
        'PBL',
        'ART',
        'KITCHEN',
        'LECTURE'
    ]

    # 조건을 적용하여 'Category' 열 생성
    offer['Category'] = np.select(conditions, choices, default=np.nan)
    
    return offer

file_path = r"C:\Users\light\Desktop\CSD - Course Offer.xlsx"  # Windows 경로 예시
offer = preprocess_offer(file_path)

offer

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer,Category
6,AB443,SABE,LECTURE,25,180,NURUL ANIDA MOHAMAD,LECTURE
7,AB516,SABE,LECTURE FATHER CLASS,50,450,LIM KER CHWING,LECTURE
8,AB533,SABE,LECTURE FATHER CLASS,40,180,ASST. PROF. IDR BAIZURA HANIM BT BIDIN,LECTURE
10,AB618,SABE,LECTURE 2,20,240,NURUL ANIDA MOHAMAD,LECTURE
11,AB618,SABE,LECTURE 1,20,210,NURUL ANIDA MOHAMAD,LECTURE
...,...,...,...,...,...,...,...
1152,SP206,FOSSLA,LECTURE 1,20,180,NURUL HIDAYAH BT MOHD SA'AT,LECTURE
1153,SP206,FOSSLA,LECTURE 2,20,180,NURUL HIDAYAH BT MOHD SA'AT,LECTURE
1154,SP208,FOSSLA,LECTURE,5,120,ARMAN IMRAN ASHOK,LECTURE
1155,SP208,FOSSLA,LECTURE,5,180,ARMAN IMRAN ASHOK,LECTURE


In [248]:
import pandas as pd

# 엑셀 파일의 절대 경로 또는 주피터 노트북 기준 상대 경로를 지정합니다.
file_path = r"C:\Users\light\Desktop\CSD - Course Offer.xlsx"  # Windows 경로 예시

# 엑셀 파일을 DataFrame으로 불러옵니다.
offer = pd.read_excel(file_path)

offer.drop(columns=['Unnamed: 0'], inplace=True)
offer.drop(offer.index[1157], axis=0, inplace=True)
offer['Capacity'] = offer['Capacity'].astype(int)
offer['Min Per Session'] = offer['Min Per Session'].astype(int)

total_time = 134790

offer

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
0,AB243,SABE,Lecture Combined To BEI1083/Lecture,5,180,Nurul Anida Mohamad
1,AB316,SABE,Lecture Combined To BEI2017/Lecture,5,450,Dr. Wong Leong Yee
2,AB343,SABE,Lecture Combined To BEI2033/Lecture,5,180,Humanson Gimfil
3,AB416,SABE,Lecture 1 Combined To BEI2067/Lecture 1,10,210,Asst. Prof. IDr Baizura Hanim Bt Bidin
4,AB416,SABE,Lecture 2 Combined To BEI2067/Lecture 2,10,240,Asst. Prof. IDr Baizura Hanim Bt Bidin
...,...,...,...,...,...,...
1152,SP206,FOSSLA,Lecture (S),20,180,Nurul Hidayah Bt Mohd Sa'at
1153,SP206,FOSSLA,Lecture (S),20,180,Nurul Hidayah Bt Mohd Sa'at
1154,SP208,FOSSLA,Lecture (S),5,120,Arman Imran Ashok
1155,SP208,FOSSLA,Lecture (S),5,180,Arman Imran Ashok


In [249]:
#전체 데이터를 대문자로 바꾸는 코드
offer['Session'] = offer['Session'].str.upper()
offer['Lecturer'] = offer['Lecturer'].str.upper()
offer['CourseCode'] = offer['CourseCode'].str.upper()
offer['FacultyCode'] = offer['FacultyCode'].str.upper()

offer

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
0,AB243,SABE,LECTURE COMBINED TO BEI1083/LECTURE,5,180,NURUL ANIDA MOHAMAD
1,AB316,SABE,LECTURE COMBINED TO BEI2017/LECTURE,5,450,DR. WONG LEONG YEE
2,AB343,SABE,LECTURE COMBINED TO BEI2033/LECTURE,5,180,HUMANSON GIMFIL
3,AB416,SABE,LECTURE 1 COMBINED TO BEI2067/LECTURE 1,10,210,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
4,AB416,SABE,LECTURE 2 COMBINED TO BEI2067/LECTURE 2,10,240,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
...,...,...,...,...,...,...
1152,SP206,FOSSLA,LECTURE (S),20,180,NURUL HIDAYAH BT MOHD SA'AT
1153,SP206,FOSSLA,LECTURE (S),20,180,NURUL HIDAYAH BT MOHD SA'AT
1154,SP208,FOSSLA,LECTURE (S),5,120,ARMAN IMRAN ASHOK
1155,SP208,FOSSLA,LECTURE (S),5,180,ARMAN IMRAN ASHOK


In [250]:
#전체 데이터에서 (s)를 삭제하는 코드
offer['Session'] = offer['Session'].str.replace(r'\(S\)', '', regex=True)
offer

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
0,AB243,SABE,LECTURE COMBINED TO BEI1083/LECTURE,5,180,NURUL ANIDA MOHAMAD
1,AB316,SABE,LECTURE COMBINED TO BEI2017/LECTURE,5,450,DR. WONG LEONG YEE
2,AB343,SABE,LECTURE COMBINED TO BEI2033/LECTURE,5,180,HUMANSON GIMFIL
3,AB416,SABE,LECTURE 1 COMBINED TO BEI2067/LECTURE 1,10,210,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
4,AB416,SABE,LECTURE 2 COMBINED TO BEI2067/LECTURE 2,10,240,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
...,...,...,...,...,...,...
1152,SP206,FOSSLA,LECTURE,20,180,NURUL HIDAYAH BT MOHD SA'AT
1153,SP206,FOSSLA,LECTURE,20,180,NURUL HIDAYAH BT MOHD SA'AT
1154,SP208,FOSSLA,LECTURE,5,120,ARMAN IMRAN ASHOK
1155,SP208,FOSSLA,LECTURE,5,180,ARMAN IMRAN ASHOK


In [251]:
# "COMBINED TO /"를 복사하여 error 테이블에 저장. 이후 원본 데이터에서 삭제.
mask = offer['Session'].str.contains("COMBINED TO /", na=False)

# error 데이터프레임에 해당 행들을 복사하여 저장합니다.
error = offer[mask].copy()

# 원본 offer 데이터프레임에서 해당 행들을 삭제합니다.
offer.drop(offer[mask].index, inplace=True)

error

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
80,BAC1014,FAS,COMBINED TO /,5,180,NaN
162,BBA1134,FBM,COMBINED TO /,20,180,NaN
863,EA103,FETBE,COMBINED TO /,2,60,NaN
864,EA103,FETBE,COMBINED TO /,2,60,NaN
1050,MS201,FAS,COMBINED TO /,10,120,NaN
1051,MS201,FAS,COMBINED TO /,10,60,NaN
1052,MS201,FAS,COMBINED TO /,10,60,NaN
1053,MS201,FAS,COMBINED TO /,10,120,NaN


In [252]:
offer

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
0,AB243,SABE,LECTURE COMBINED TO BEI1083/LECTURE,5,180,NURUL ANIDA MOHAMAD
1,AB316,SABE,LECTURE COMBINED TO BEI2017/LECTURE,5,450,DR. WONG LEONG YEE
2,AB343,SABE,LECTURE COMBINED TO BEI2033/LECTURE,5,180,HUMANSON GIMFIL
3,AB416,SABE,LECTURE 1 COMBINED TO BEI2067/LECTURE 1,10,210,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
4,AB416,SABE,LECTURE 2 COMBINED TO BEI2067/LECTURE 2,10,240,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
...,...,...,...,...,...,...
1152,SP206,FOSSLA,LECTURE,20,180,NURUL HIDAYAH BT MOHD SA'AT
1153,SP206,FOSSLA,LECTURE,20,180,NURUL HIDAYAH BT MOHD SA'AT
1154,SP208,FOSSLA,LECTURE,5,120,ARMAN IMRAN ASHOK
1155,SP208,FOSSLA,LECTURE,5,180,ARMAN IMRAN ASHOK


In [253]:
#"GROUP A", "GROUP B", "GROUP C" 등 해당하는 행을 찾고, Session 값을 "LECTURE"로 변경
offer.loc[offer['Session'].str.contains(r'^GROUP\s+[A-Z]$', na=False), 'Session'] = 'LECTURE'

# 결과 확인
offer.loc[[1027]]


,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
1027,MPU3221,FOSSLA,LECTURE,300,60,DR. SITI NUR AAFIFAH HASHIM


In [254]:
#중복된 행들에 숫자를 붙여서 다른 수업임을 명시. 이때 이미 숫자가 붙어있는상태로 중복이라면, 해당 숫자를 기준으로 중복을 처리함.
import re

group_cols = ['CourseCode', 'FacultyCode', 'Session', 'Capacity', 'Min Per Session', 'Lecturer']

# 그룹별로 중복된 행을 처리합니다.
for _, group in offer.groupby(group_cols):
    if len(group) > 1:  # 중복 그룹인 경우
        # 원래 Session 값을 앞뒤 공백 제거 후 사용합니다.
        orig_session = group.iloc[0]['Session'].strip()
        # Session 값이 숫자로 끝나는지 검사 (예: "TUTORIAL 1")
        m = re.search(r'^(.*?)(\d+)$', orig_session)
        if m:
            base = m.group(1).strip()  # 문자 부분 (예: "TUTORIAL")
            start_num = int(m.group(2))  # 기존에 붙은 숫자 (예: 1)
        else:
            base = orig_session
            start_num = 1
        
        # 그룹 내 각 행에 대해 순차적으로 번호를 붙여 Session 값을 변경합니다.
        for offset, idx in enumerate(group.index):
            new_session = f"{base} {start_num + offset}"
            offer.at[idx, 'Session'] = new_session

offer.loc[[166]]

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
166,BBA1703,FBM,TUTORIAL 2,40,60,BRAHAM RAHUL RAM A/L JAMNADAS


In [255]:
import pandas as pd

# (초기 전제: offer DataFrame과 error DataFrame이 존재함; error가 없으면 빈 DataFrame 생성)
if 'error' not in globals():
    error = pd.DataFrame()

### Step 1. offer에서 "COMBINED TO" 행 추출 → child_class로 저장, offer에서는 제거
child_class = offer[offer['Session'].str.contains('COMBINED TO', na=False)].copy()
offer.drop(child_class.index, inplace=True)

### Step 2. child_class의 Session 열에서 과목코드 추출
child_class['extracted_code'] = child_class['Session'].str.extract(r'COMBINED TO\s+([A-Z0-9]+)', expand=False)

### Step 3. child_class 내에서 오류 및 정상/후손 분리

# (A) "자기 자신 참조" 조건: 추출한 코드가 해당 행의 CourseCode와 같으면 → 오류
self_ref = child_class['extracted_code'] == child_class['CourseCode']

# (B) 정상(child) 조건: 자기 자신 참조가 아니고, 추출한 코드가 offer의 CourseCode에 존재하는 경우
valid_child_mask = (~self_ref) & (child_class['extracted_code'].isin(offer['CourseCode']))

# (C) 후손(grandchild) 후보: 자기 자신 참조가 아니고, 추출한 코드가 offer에는 없지만, child_class의 CourseCode에는 존재하는 경우  
#     (즉, 부모(대상 CourseCode)가 offer에 없으므로 자식끼리 연결되어 있음을 의미)
valid_grand_mask = (~self_ref) & (~child_class['extracted_code'].isin(offer['CourseCode'])) & \
                    (child_class['extracted_code'].isin(child_class['CourseCode']))

# (D) 오류 조건:  
#     - 자기 자신 참조인 경우  
#     - 또는 추출한 코드가 offer에도 child_class에도 존재하지 않는 경우
error_mask = self_ref | ((~child_class['extracted_code'].isin(offer['CourseCode'])) & 
                           (~child_class['extracted_code'].isin(child_class['CourseCode'])))

# child_class 오류 처리: 추출
child_error = child_class[error_mask].copy()

# 정상 child_class (valid_child) 추출
valid_child = child_class[valid_child_mask].copy()

# 후보 후손(grandchild) 추출
candidate_grand = child_class[valid_grand_mask].copy()

### Step 4. grand_child_class 내에서 추가 오류 처리
# 오류 조건 (grandchild): 자기 자신 참조 → 오류
grand_self_ref = candidate_grand['extracted_code'] == candidate_grand['CourseCode']
error_grand = candidate_grand[grand_self_ref].copy()

# 최종 grand_child_class: 후보에서 자기 자신 참조 오류 제거
grand_child_class = candidate_grand[~grand_self_ref].copy()

# 최종 error: 기존 child 오류와 후손에서 발생한 오류를 모두 누적
error_df = pd.concat([child_error, error_grand], ignore_index=True)
error = pd.concat([error, error_df], ignore_index=True)

### 최종 정상 데이터
# child_class는 valid_child로 업데이트 (offer의 부모가 존재하는 경우)
child_class = valid_child.copy()
# grand_child_class는 위에서 구한 대로

In [256]:
import pandas as pd

# error DataFrame이 없으면 빈 DataFrame 생성
if 'error' not in globals():
    error = pd.DataFrame()

##############################################
# 교차검증 1: child_class에서 자기 자신 참조하는 행 처리
##############################################
child_self_ref_mask = (child_class['extracted_code'] == child_class['CourseCode'])
child_self_ref_errors = child_class[child_self_ref_mask].copy()
error = pd.concat([error, child_self_ref_errors], ignore_index=True)
child_class = child_class[~child_self_ref_mask].copy()

##############################################
# 교차검증 2: child_class에서 추출한 과목코드가 offer의 CourseCode에 없음(부모 없음)
##############################################
child_missing_parent_mask = ~child_class['extracted_code'].isin(offer['CourseCode'])
child_missing_parent_errors = child_class[child_missing_parent_mask].copy()
error = pd.concat([error, child_missing_parent_errors], ignore_index=True)
child_class = child_class[~child_missing_parent_mask].copy()

##############################################
# 교차검증 3: offer에서 자기 자신 참조하는 행 처리
# (offer에 COMBINED TO가 남아있다면 해당 행에 대해 extracted_code를 추출)
##############################################
offer_combined_mask = offer['Session'].str.contains('COMBINED TO', na=False)
if offer_combined_mask.any():
    offer.loc[offer_combined_mask, 'extracted_code'] = offer.loc[offer_combined_mask, 'Session'].str.extract(r'COMBINED TO\s+([A-Z0-9]+)', expand=False)
    offer_self_ref_mask = (offer['extracted_code'] == offer['CourseCode'])
    offer_self_ref_errors = offer[offer_self_ref_mask].copy()
    error = pd.concat([error, offer_self_ref_errors], ignore_index=True)
    offer.drop(offer[offer_self_ref_mask].index, inplace=True)

##############################################
# 교차검증 4: grand_child_class에서 자기 자신 참조하는 행 처리
##############################################
grand_self_ref_mask = (grand_child_class['extracted_code'] == grand_child_class['CourseCode'])
grand_self_ref_errors = grand_child_class[grand_self_ref_mask].copy()
error = pd.concat([error, grand_self_ref_errors], ignore_index=True)
grand_child_class = grand_child_class[~grand_self_ref_mask].copy()

##############################################
# 교차검증 5: grand_child_class에서 추출한 과목코드가 offer의 CourseCode에 없음(부모 없음)
##############################################
grand_missing_parent_mask = ~grand_child_class['extracted_code'].isin(offer['CourseCode'])
grand_missing_parent_errors = grand_child_class[grand_missing_parent_mask].copy()
error = pd.concat([error, grand_missing_parent_errors], ignore_index=True)
grand_child_class = grand_child_class[~grand_missing_parent_mask].copy()

##############################################
# 교차검증 6: 부모(offer 또는 child_class)에서 드랍된 행과 연결된 자식(또는 손자) 행 처리
# 드랍된 행들의 CourseCode를 모아서, 이 코드를 extracted_code로 사용하는 행이 있다면 error로 처리
##############################################
# 지금까지 error에 추가된 행들의 CourseCode를 모읍니다.
dropped_codes = set(error['CourseCode'].unique())

# child_class에서 연결된 자식 행 검사
child_linked_mask = child_class['extracted_code'].isin(dropped_codes)
child_linked_errors = child_class[child_linked_mask].copy()
error = pd.concat([error, child_linked_errors], ignore_index=True)
child_class = child_class[~child_linked_mask].copy()

# grand_child_class에서 연결된 손자 행 검사
grand_linked_mask = grand_child_class['extracted_code'].isin(dropped_codes)
grand_linked_errors = grand_child_class[grand_linked_mask].copy()
error = pd.concat([error, grand_linked_errors], ignore_index=True)
grand_child_class = grand_child_class[~grand_linked_mask].copy()

##############################################
# 최종 결과 출력 (각 DataFrame의 shape 등 확인)
##############################################
print("offer 남은 행:", offer.shape)
print("child_class 남은 행:", child_class.shape)
print("grand_child_class 남은 행:", grand_child_class.shape)
print("error 행:", error.shape)


offer 남은 행: (808, 6)
child_class 남은 행: (183, 7)
grand_child_class 남은 행: (0, 7)
error 행: (166, 7)


In [257]:
child_class

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer,extracted_code
0,AB243,SABE,LECTURE COMBINED TO BEI1083/LECTURE,5,180,NURUL ANIDA MOHAMAD,BEI1083
1,AB316,SABE,LECTURE COMBINED TO BEI2017/LECTURE,5,450,DR. WONG LEONG YEE,BEI2017
2,AB343,SABE,LECTURE COMBINED TO BEI2033/LECTURE,5,180,HUMANSON GIMFIL,BEI2033
3,AB416,SABE,LECTURE 1 COMBINED TO BEI2067/LECTURE 1,10,210,ASST. PROF. IDR BAIZURA HANIM BT BIDIN,BEI2067
4,AB416,SABE,LECTURE 2 COMBINED TO BEI2067/LECTURE 2,10,240,ASST. PROF. IDR BAIZURA HANIM BT BIDIN,BEI2067
...,...,...,...,...,...,...,...
1049,MS201,FAS,LECTURE 2 COMBINED TO BAA2023/LECTURE 2,10,60,ASST. PROF. DR. CHEW LI LEE,BAA2023
1054,MST1014B,FOSSLA,LECTURE COMBINED TO PST1014B/LECTURE 1,10,240,ASST. PROF. DR. ZAIDA MUSTAFA,PST1014B
1055,MST1014B,FOSSLA,LECTURE COMBINED TO PST1014B/LECTURE 2,10,240,ASST. PROF. DR. ZAIDA MUSTAFA,PST1014B
1056,MST1024B,FOSSLA,LECTURE COMBINED TO PST1024B/LECTURE 1,10,240,ASST. PROF. DR. SIMRANJEET KAUR JUDGE A/P CHAR...,PST1024B


In [258]:
# child_class의 각 행을 순회하며 조건에 맞는 offer 행의 Capacity에 child_class의 Capacity를 합산합니다.
for idx, child_row in child_class.iterrows():
    # 조건: offer의 CourseCode, Min Per Session, Lecturer가 child_row의 extracted_code, Min Per Session, Lecturer와 일치
    mask = (
        (offer['CourseCode'] == child_row['extracted_code']) &
        (offer['Min Per Session'] == child_row['Min Per Session']) &
        (offer['Lecturer'] == child_row['Lecturer'])
    )
    # 일치하는 행이 있다면, offer의 Capacity에 child_row의 Capacity를 더함
    if mask.any():
        offer.loc[mask, 'Capacity'] += child_row['Capacity']

# 결과 확인: 업데이트된 offer 데이터프레임의 Capacity 값
offer

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
6,AB443,SABE,LECTURE,25,180,NURUL ANIDA MOHAMAD
7,AB516,SABE,LECTURE FATHER CLASS,50,450,LIM KER CHWING
8,AB533,SABE,LECTURE FATHER CLASS,40,180,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
10,AB618,SABE,LECTURE 2,20,240,NURUL ANIDA MOHAMAD
11,AB618,SABE,LECTURE 1,20,210,NURUL ANIDA MOHAMAD
...,...,...,...,...,...,...
1152,SP206,FOSSLA,LECTURE 1,20,180,NURUL HIDAYAH BT MOHD SA'AT
1153,SP206,FOSSLA,LECTURE 2,20,180,NURUL HIDAYAH BT MOHD SA'AT
1154,SP208,FOSSLA,LECTURE,5,120,ARMAN IMRAN ASHOK
1155,SP208,FOSSLA,LECTURE,5,180,ARMAN IMRAN ASHOK


In [302]:
import numpy as np

# 기존 조건
conditions = [
    offer['Session'].str.contains('LECTURE & TUTORIAL', na=False, case=False),
    offer['Session'].str.contains('TUTORIAL', na=False, case=False),
    offer['Session'].str.contains('LECTURE', na=False, case=False),
    offer['Session'].str.contains('LAB', na=False, case=False),
    offer['Session'].str.contains('PBL 1', na=False, case=False),
    offer['Session'].str.contains('HAND DRAWING|CAD DRAWING 2|CAD DRAWING 1', na=False, case=False),
    offer['Session'].str.contains('KITCHEN', na=False, case=False),
    offer['Session'].str.contains('OPTOM CLINIC|SOO - EXAM CLINIC', na=False, case=False)
]

# 기존 선택지 + 새 선택지
choices = [
    'GENERAL',
    'TUTORIAL',
    'LECTURE',
    'LAB',
    'PBL',
    'ART',
    'KITCHEN',
    'LECTURE'
]

# 조건을 적용하여 'Category' 열 생성
offer['Category'] = np.select(conditions, choices, default=np.nan)

offer


,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer,Category
6,AB443,SABE,LECTURE,25,180,NURUL ANIDA MOHAMAD,LECTURE
7,AB516,SABE,LECTURE FATHER CLASS,50,450,LIM KER CHWING,LECTURE
8,AB533,SABE,LECTURE FATHER CLASS,40,180,ASST. PROF. IDR BAIZURA HANIM BT BIDIN,LECTURE
10,AB618,SABE,LECTURE 2,20,240,NURUL ANIDA MOHAMAD,LECTURE
11,AB618,SABE,LECTURE 1,20,210,NURUL ANIDA MOHAMAD,LECTURE
...,...,...,...,...,...,...,...
1152,SP206,FOSSLA,LECTURE 1,20,180,NURUL HIDAYAH BT MOHD SA'AT,LECTURE
1153,SP206,FOSSLA,LECTURE 2,20,180,NURUL HIDAYAH BT MOHD SA'AT,LECTURE
1154,SP208,FOSSLA,LECTURE,5,120,ARMAN IMRAN ASHOK,LECTURE
1155,SP208,FOSSLA,LECTURE,5,180,ARMAN IMRAN ASHOK,LECTURE


In [303]:
result = offer[offer['CourseCode'].str.lower() == 'bp132']
result


,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer,Category
541,BP132,FPS,LECTURE 1,60,60,PROF. TS. DR. LEE MING TATT,LECTURE
542,BP132,FPS,LECTURE 2,60,60,PROF. TS. DR. LEE MING TATT,LECTURE
543,BP132,FPS,LECTURE 3,60,60,PROF. TS. DR. LEE MING TATT,LECTURE
544,BP132,FPS,TUTORIAL 1,60,60,PROF. TS. DR. LEE MING TATT,TUTORIAL
545,BP132,FPS,LECTURE & TUTORIAL,60,60,PROF. TS. DR. LEE MING TATT,GENERAL


In [304]:
mask = ~offer['Category'].str.contains('TUTORIAL|LECTURE|LAB', na=False)
offer[mask]


,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer,Category
537,BP112,FPS,LECTURE & TUTORIAL,140,60,PROFESSOR DR. SAAD TAYYAB,GENERAL
545,BP132,FPS,LECTURE & TUTORIAL,60,60,PROF. TS. DR. LEE MING TATT,GENERAL
548,BP172,FPS,LECTURE & TUTORIAL,60,60,VITHYAH A/P NADARAJA,GENERAL
568,BP266,FPS,LECTURE & TUTORIAL,60,60,SABREEN YOUSIF MOHAMMED ALHASSAN NASR,GENERAL
653,BPC2122,FPS,LECTURE & TUTORIAL,20,60,ASSOCIATE PROFESSOR DR. MAI CHUN WAI,GENERAL
839,DEX1034,FETBE,HAND DRAWING,15,120,TS. AMAR RIDZUAN ABD HAMID,ART
841,DEX1034,FETBE,CAD DRAWING 2,15,120,TS. MUHAMAD FALIQ MOHAMAD NAZER,ART
842,DEX1034,FETBE,CAD DRAWING 1,15,120,TS. MUHAMAD FALIQ MOHAMAD NAZER,ART
853,DPB1014,FHTM,KITCHEN PRACTICAL,10,360,SH MARIA SAHILA BT SYED ALI HASSAN,KITCHEN
854,DPB1024,FHTM,LECTURE & TUTORIAL,10,180,NURSYAFIQAH RAMLI,GENERAL


# classroom

In [2]:
import pandas as pd
import plotly.express as px

df = pd.read_excel(r"C:\Users\light\Desktop\CSD - Resource Room.xlsx")

Resource_Room = pd.DataFrame(df)
Resource_Room = Resource_Room.drop(labels='Campus', axis=1)
Resource_Room = Resource_Room.drop(labels='Workshop', axis=1)
Resource_Room = Resource_Room[~Resource_Room.apply(lambda row: row.astype(str).str.contains('IMus', case=False, na=False).any(), axis=1)]
Resource_Room = Resource_Room[~(Resource_Room[['Lecture', 'Tutorial', 'Lab']].eq('N').all(axis=1))]
Resource_Room = Resource_Room.fillna('N')
Resource_Room['Capacity'] = Resource_Room['Capacity'].replace('N', '20')
Resource_Room = Resource_Room.drop(Resource_Room[Resource_Room['Capacity'] == 0].index)

print(Resource_Room[Resource_Room['Capacity'] == 'N'])

#조건에 따라 Gerneral한 경우로 분류
conditions = [
    ['Y', 'Y', 'Y']
]
columns_to_check = ['Lecture', 'Tutorial', 'Lab']
General = Resource_Room[Resource_Room[columns_to_check].apply(tuple, axis=1).isin(map(tuple, conditions))]
General.info()

#Gerneral하지 않은 경우를 뺀 나머지 데이터프레임 생성
conditions_to_exclude = [
    ['Y', 'Y', 'N'],
    ['N', 'Y', 'Y']
]
columns_to_check = ['Lecture', 'Tutorial', 'Lab']

#G_Resource_Room = Resource_Room[~Resource_Room[columns_to_check].apply(lambda row: tuple(row) in conditions_to_exclude, axis=1)]
#G_Resource_Room.info()

etc_Room = Resource_Room[~Resource_Room[columns_to_check].apply(tuple, axis=1).isin(map(tuple, conditions))]

#G_Resource_Room_IMus = G_Resource_Room[G_Resource_Room['Description'].str.contains('IMus', na=False) | G_Resource_Room['Resource Code'].str.contains('IMus', na=False)]
Kitchen = etc_Room[etc_Room.apply(lambda row: row.astype(str).str.contains('Kitchen', regex=False).any(), axis=1)]
Design = etc_Room[etc_Room['Description'].str.contains('DESIGN', case=False, na=False)]
df_combined = pd.concat([Kitchen, Design])
LL = etc_Room[~etc_Room.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))]

#조건에 따라 Gerneral한 경우로 분류
conditions = [
    ['N', 'N', 'Y']
]
columns_to_check = ['Lecture', 'Tutorial', 'Lab']
Lab = LL[LL[columns_to_check].apply(tuple, axis=1).isin(map(tuple, conditions)) & ~LL.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))]

#조건에 따라 Gerneral한 경우로 분류
conditions = [
    ['Y', 'N', 'N']
]
columns_to_check = ['Lecture', 'Tutorial', 'Lab']
Lecture = LL[LL[columns_to_check].apply(tuple, axis=1).isin(map(tuple, conditions)) & ~LL.apply(tuple, axis=1).isin(df_combined.apply(tuple, axis=1))]

df_combined = pd.concat([Lab, Lecture])
ETC = LL[~LL.apply(tuple, 1).isin(df_combined.apply(tuple, 1))]

import pandas as pd

General['Type'] = 'General'
Lab['Type'] = 'Lab'
Kitchen['Type'] = 'Kitchen'
Design['Type'] = 'Art'
Lecture['Type'] = 'Lecture'
ETC['Type'] = 'Etc'

df_final = pd.concat([General, ETC, Kitchen, Design, Lab, Lecture], ignore_index=True)
df_final = df_final.drop_duplicates(keep='first')

df_final.loc[df_final['Resource Code'].str.contains('PBL', na=False), 'Type'] = 'PBL'
df_final[df_final['Type'] == 'PBL']
df_final['Resource Code'] = df_final['Resource Code'].str.upper()
df_final['Description'] = df_final['Description'].str.upper()
df_final['Resource Status'] = df_final['Resource Status'].str.upper()
df_final['Type'] = df_final['Type'].str.upper()
df_final.rename(columns={'Type': 'Category'}, inplace=True)
classroom = df_final.copy()
classroom

Empty DataFrame
Columns: [Resource Code, Description, Capacity, Lecture, Tutorial, Lab, Resource Status]
Index: []
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 16 to 171
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Resource Code    7 non-null      object
 1   Description      7 non-null      object
 2   Capacity         7 non-null      int64 
 3   Lecture          7 non-null      object
 4   Tutorial         7 non-null      object
 5   Lab              7 non-null      object
 6   Resource Status  7 non-null      object
dtypes: int64(1), object(6)
memory usage: 448.0+ bytes


C:\Users\light\AppData\Local\Temp\ipykernel_7096\3586962336.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  General['Type'] = 'General'
C:\Users\light\AppData\Local\Temp\ipykernel_7096\3586962336.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Lab['Type'] = 'Lab'
C:\Users\light\AppData\Local\Temp\ipykernel_7096\3586962336.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,Resource Code,Description,Capacity,Lecture,Tutorial,Lab,Resource Status,Category
0,MPH - GBS,MPH - [MULTIPURPOSE HALL],100,Y,Y,Y,ACTIVE,GENERAL
1,C3M04B,C3M04 - [C3M04B-SABE COMPUTER LAB],30,Y,Y,Y,ACTIVE,GENERAL
2,C3M05,C3M05 - [C3M05-COMPUTER LAB],30,Y,Y,Y,ACTIVE,GENERAL
3,C314,C314 - [MICROTEACHING LAB],30,Y,Y,Y,ACTIVE,GENERAL
4,"ANALYTICAL & SENSING, INSTRUMENTATIONS & CONTR...","ANALYTICAL & SENSING, INSTRUMENTATIONS & CONTR...",30,Y,Y,Y,ACTIVE,GENERAL
...,...,...,...,...,...,...,...,...
179,PETROL CHEMICAL LAB,PETROL CHEMICAL STATISTICAL ANALYSIS LABORATORY,15,N,N,Y,ACTIVE,LAB
180,ADV CHEM LAB,ADVANCE CHEMICAL TECHNOLOGY LABORATORY,15,N,N,Y,ACTIVE,LAB
181,POSTGRADUATE RESEARCH & SYSTEM ANALYTICAL LABO...,POSTGRADUATE RESEARCH & SYSTEM ANALYTICAL LABO...,15,N,N,Y,ACTIVE,LAB
182,FASMR2,FAS MR 2,10,N,N,Y,ACTIVE,LAB


# Model 

In [8]:
import pandas as pd
import numpy as np
import random
import math

# -------------------------------
# 1. Offer 데이터 전처리: 세션 분할
# -------------------------------
def split_sessions(offer):
    """
    offer 데이터에서 'Min Per Session'이 180분을 초과하는 경우,
    180분 이하 단위로 분할하여 여러 세션(row)으로 반환합니다.
    예) 450분 → 180, 180, 90 (각각 _part1, _part2, _part3 로 Session 구분)
    """
    sessions = []
    for idx, row in offer.iterrows():
        total_minutes = row['Min Per Session']
        if total_minutes <= 180:
            sessions.append(row.copy())
        else:
            remaining = total_minutes
            part = 1
            while remaining > 0:
                new_row = row.copy()
                chunk = min(180, remaining)
                new_row['Min Per Session'] = chunk
                new_row['Session'] = f"{row['Session']}_part{part}"
                sessions.append(new_row)
                remaining -= chunk
                part += 1
    return pd.DataFrame(sessions)

# offer 데이터가 이미 로드되어 있다고 가정
# 인덱스 재설정 추가!
df_sessions = split_sessions(offer).reset_index(drop=True)
print("총 세션 수 (분할 후):", len(df_sessions))


# -------------------------------
# 2. 시간 슬롯(Time Slot) 정의 (월~금, 08시~21시, 1시간 단위)
# -------------------------------
days = ["Mon", "Tue", "Wed", "Thu", "Fri"]

def get_valid_start_hours(duration_hours):
    """
    주어진 세션의 지속시간(duration_hours)이 있을 때,
    해당 세션이 하루 08:00~21:00 내에 배정될 수 있는 시작시간 목록을 반환.
    (start_hour + duration_hours <= 21)
    """
    return [h for h in range(8, 22) if h + duration_hours <= 21]


# -------------------------------
# 3. 랜덤 서치: 스케줄(시간표) 생성 함수
# -------------------------------
def generate_random_schedule(df_sessions, classroom):
    """
    각 세션에 대해 다음을 수행:
      - 해당 세션의 요구조건(수용인원, 강의/튜토리얼 vs. Lab)을 만족하는 강의실 후보에서 랜덤 선택
      - 월~금 중 랜덤 요일 선택
      - 세션의 지속시간(분)을 1시간 단위(올림)로 계산 후, 시작시간 후보 중 랜덤 선택
    """
    schedule = []
    for idx, session in df_sessions.iterrows():
        # 1. 세션의 지속시간(시간 단위): 1시간 단위 올림 (예: 90분 → 2시간, 180분 → 3시간)
        duration_hours = math.ceil(session['Min Per Session'] / 60)
        
        # 2. 강의실 후보 선정: 용량 조건 확인
        valid_rooms = classroom[classroom['Capacity'] >= session['Capacity']]
        
        # 3. 강의실 유형 조건
        # Lab 강의는 반드시 Lab 강의실에 배정 (그 외는 일반 강의실: GENERAL 또는 LECTURE)
        session_cat = session['Category'].upper()
        if session_cat == "LAB":
            valid_rooms = valid_rooms[valid_rooms['Category'].str.upper() == "LAB"]
        else:
            valid_rooms = valid_rooms[valid_rooms['Category'].str.upper().isin(["GENERAL", "LECTURE"])]
        
        # 4. 강의실 후보가 없으면 None 할당
        if valid_rooms.empty:
            assigned_room = None
        else:
            assigned_room = valid_rooms.sample(1)['Resource Code'].values[0]
        
        # 5. 요일과 시작시간 랜덤 선택
        assigned_day = random.choice(days)
        valid_hours = get_valid_start_hours(duration_hours)
        if not valid_hours:
            assigned_hour = None
        else:
            assigned_hour = random.choice(valid_hours)
        
        assignment = {
            'session_index': idx,               # df_sessions의 인덱스를 저장 (후에 세부정보 참조)
            'course': session['CourseCode'],
            'lecturer': session['Lecturer'],
            'category': session['Category'],
            'required_duration': duration_hours,  # 필요한 연속 슬롯 수
            'assigned_classroom': assigned_room,
            'day': assigned_day,
            'start_hour': assigned_hour,
            'end_hour': assigned_hour + duration_hours if assigned_hour is not None else None
        }
        schedule.append(assignment)
    return schedule


# -------------------------------
# 4. 스코어링 함수: 스케줄 평가 (제약 위반에 따른 페널티 부여)
# -------------------------------
def score_schedule(schedule, df_sessions, classroom):
    """
    각 스케줄의 제약 위반 여부에 따라 패널티를 부여합니다.
    
    제약 조건:
      - 할당된 강의실이나 시작시간이 None이면 heavy penalty
      - 시간대가 08:00~21:00 내에 있어야 함
      - 강의실 용량 부족 → penalty
      - Lab 강의는 반드시 Lab 강의실에 배정되어야 함
      - 동일 요일에 동일 강의실 또는 동일 강사가 시간 겹침이 있으면 penalty
    """
    penalty = 0
    n = len(schedule)
    
    # 각 assignment를 검사
    for i in range(n):
        a = schedule[i]
        # 만약 강의실이나 시작시간이 할당되지 않았다면 heavy penalty
        if a['assigned_classroom'] is None or a['start_hour'] is None:
            penalty += 10
            continue
        
        # 시간대 범위 검사 (08시~21시)
        if a['start_hour'] < 8 or a['end_hour'] > 21:
            penalty += 5
        
        # 강의실 정보 획득
        cls = classroom[classroom['Resource Code'] == a['assigned_classroom']]
        if cls.empty:
            penalty += 5
        else:
            room_capacity = cls.iloc[0]['Capacity']
            required_capacity = df_sessions.iloc[a['session_index']]['Capacity']
            if room_capacity < required_capacity:
                penalty += 5
            
            room_cat = cls.iloc[0]['Category'].upper()
            # Lab 강의 검사
            if a['category'].upper() == "LAB" and room_cat != "LAB":
                penalty += 5
        
        # 동일 요일 내 다른 세션과 시간 겹침 검사
        for j in range(i+1, n):
            b = schedule[j]
            if a['day'] != b['day']:
                continue
            # 같은 강의실 충돌 검사
            if a['assigned_classroom'] == b['assigned_classroom']:
                if not (a['end_hour'] <= b['start_hour'] or b['end_hour'] <= a['start_hour']):
                    penalty += 3
            # 같은 강사 충돌 검사
            if a['lecturer'] == b['lecturer']:
                if not (a['end_hour'] <= b['start_hour'] or b['end_hour'] <= a['start_hour']):
                    penalty += 3
                    
    return penalty


# -------------------------------
# 5. 랜덤 서치를 통한 최적 스케줄 탐색
# -------------------------------
def random_search(df_sessions, classroom, iterations=10000):
    best_schedule = None
    best_score = float('inf')
    for it in range(iterations):
        sched = generate_random_schedule(df_sessions, classroom)
        score = score_schedule(sched, df_sessions, classroom)
        if score < best_score:
            best_score = score
            best_schedule = sched
        if it % 100 == 0:
            print(f"Iteration {it}: Best score so far = {best_score}")
    return best_schedule, best_score


# -------------------------------
# 실행 예시
# -------------------------------
# offer와 classroom 데이터는 이미 전처리되어 있다고 가정합니다.
best_schedule, best_score = random_search(df_sessions, classroom, iterations=1000)

print("\n최종 스케줄 점수:", best_score)
print("최적 스케줄 할당:")
for assignment in best_schedule:
    print(assignment)


총 세션 수 (분할 후): 870
Iteration 0: Best score so far = 10699
Iteration 100: Best score so far = 10228
Iteration 200: Best score so far = 10210
Iteration 300: Best score so far = 10105
Iteration 400: Best score so far = 10105
Iteration 500: Best score so far = 10105
Iteration 600: Best score so far = 10105
Iteration 700: Best score so far = 10105
Iteration 800: Best score so far = 10105
Iteration 900: Best score so far = 10105

최종 스케줄 점수: 10105
최적 스케줄 할당:
{'session_index': 0, 'course': 'AB443', 'lecturer': 'NURUL ANIDA MOHAMAD', 'category': 'LECTURE', 'required_duration': 3, 'assigned_classroom': 'ANALYTICAL & SENSING, INSTRUMENTATIONS & CONTROL SYSTEM LABORATORY', 'day': 'Fri', 'start_hour': 11, 'end_hour': 14}
{'session_index': 1, 'course': 'AB516', 'lecturer': 'LIM KER CHWING', 'category': 'LECTURE', 'required_duration': 3, 'assigned_classroom': 'MPH - GBS', 'day': 'Mon', 'start_hour': 16, 'end_hour': 19}
{'session_index': 2, 'course': 'AB516', 'lecturer': 'LIM KER CHWING', 'category': 

In [10]:
import pandas as pd
import numpy as np
import random
import math

# -------------------------------
# 1. Offer 데이터 전처리: 세션 분할
# -------------------------------
def split_sessions(offer):
    """
    offer 데이터에서 'Min Per Session'이 180분을 초과하는 경우,
    180분 이하 단위로 분할하여 여러 세션(row)으로 반환합니다.
    예) 450분 → 180, 180, 90 (각각 _part1, _part2, _part3 로 Session 구분)
    """
    sessions = []
    for idx, row in offer.iterrows():
        total_minutes = row['Min Per Session']
        if total_minutes <= 180:
            sessions.append(row.copy())
        else:
            remaining = total_minutes
            part = 1
            while remaining > 0:
                new_row = row.copy()
                chunk = min(180, remaining)
                new_row['Min Per Session'] = chunk
                new_row['Session'] = f"{row['Session']}_part{part}"
                sessions.append(new_row)
                remaining -= chunk
                part += 1
    return pd.DataFrame(sessions)

# offer 데이터가 이미 로드되어 있다고 가정
# 인덱스 재설정 추가!
df_sessions = split_sessions(offer).reset_index(drop=True)
print("총 세션 수 (분할 후):", len(df_sessions))


# -------------------------------
# 2. 시간 슬롯(Time Slot) 정의 (월~금, 08시~21시, 1시간 단위)
# -------------------------------
days = ["Mon", "Tue", "Wed", "Thu", "Fri"]

def get_valid_start_hours(duration_hours):
    """
    주어진 세션의 지속시간(duration_hours)이 있을 때,
    해당 세션이 하루 08:00~21:00 내에 배정될 수 있는 시작시간 목록을 반환.
    (start_hour + duration_hours <= 21)
    """
    return [h for h in range(8, 22) if h + duration_hours <= 21]


# -------------------------------
# 3. 랜덤 서치: 스케줄(시간표) 생성 함수
# -------------------------------
def generate_random_schedule(df_sessions, classroom):
    """
    각 세션에 대해 다음을 수행:
      - 해당 세션의 요구조건(수용인원, 강의/튜토리얼 vs. Lab)을 만족하는 강의실 후보에서 랜덤 선택
      - 월~금 중 랜덤 요일 선택
      - 세션의 지속시간(분)을 1시간 단위(올림)로 계산 후, 시작시간 후보 중 랜덤 선택
    """
    schedule = []
    for idx, session in df_sessions.iterrows():
        # 1. 세션의 지속시간(시간 단위): 1시간 단위 올림 (예: 90분 → 2시간, 180분 → 3시간)
        duration_hours = math.ceil(session['Min Per Session'] / 60)
        
        # 2. 강의실 후보 선정: 용량 조건 확인
        valid_rooms = classroom[classroom['Capacity'] >= session['Capacity']]
        
        # 3. 강의실 유형 조건
        # Lab 강의는 반드시 Lab 강의실에 배정 (그 외는 일반 강의실: GENERAL 또는 LECTURE)
        session_cat = session['Category'].upper()
        if session_cat == "LAB":
            valid_rooms = valid_rooms[valid_rooms['Category'].str.upper() == "LAB"]
        else:
            valid_rooms = valid_rooms[valid_rooms['Category'].str.upper().isin(["GENERAL", "LECTURE"])]
        
        # 4. 강의실 후보가 없으면 None 할당
        if valid_rooms.empty:
            assigned_room = None
        else:
            assigned_room = valid_rooms.sample(1)['Resource Code'].values[0]
        
        # 5. 요일과 시작시간 랜덤 선택
        assigned_day = random.choice(days)
        valid_hours = get_valid_start_hours(duration_hours)
        if not valid_hours:
            assigned_hour = None
        else:
            assigned_hour = random.choice(valid_hours)
        
        assignment = {
            'session_index': idx,               # df_sessions의 인덱스를 저장 (후에 세부정보 참조)
            'course': session['CourseCode'],
            'lecturer': session['Lecturer'],
            'category': session['Category'],
            'required_duration': duration_hours,  # 필요한 연속 슬롯 수
            'assigned_classroom': assigned_room,
            'day': assigned_day,
            'start_hour': assigned_hour,
            'end_hour': assigned_hour + duration_hours if assigned_hour is not None else None
        }
        schedule.append(assignment)
    return schedule


# -------------------------------
# 4. 스코어링 함수: 스케줄 평가 (제약 위반에 따른 페널티 부여)
# -------------------------------
def score_schedule(schedule, df_sessions, classroom):
    """
    각 스케줄의 제약 위반 여부에 따라 패널티를 부여합니다.
    
    제약 조건:
      - 할당된 강의실이나 시작시간이 None이면 heavy penalty
      - 시간대가 08:00~21:00 내에 있어야 함
      - 강의실 용량 부족 → penalty
      - Lab 강의는 반드시 Lab 강의실에 배정되어야 함
      - 동일 요일에 동일 강의실 또는 동일 강사가 시간 겹침이 있으면 penalty
    """
    penalty = 0
    n = len(schedule)
    
    # 각 assignment를 검사
    for i in range(n):
        a = schedule[i]
        # 만약 강의실이나 시작시간이 할당되지 않았다면 heavy penalty
        if a['assigned_classroom'] is None or a['start_hour'] is None:
            penalty += 10
            continue
        
        # 시간대 범위 검사 (08시~21시)
        if a['start_hour'] < 8 or a['end_hour'] > 21:
            penalty += 5
        
        # 강의실 정보 획득
        cls = classroom[classroom['Resource Code'] == a['assigned_classroom']]
        if cls.empty:
            penalty += 5
        else:
            room_capacity = cls.iloc[0]['Capacity']
            required_capacity = df_sessions.iloc[a['session_index']]['Capacity']
            if room_capacity < required_capacity:
                penalty += 5
            
            room_cat = cls.iloc[0]['Category'].upper()
            # Lab 강의 검사
            if a['category'].upper() == "LAB" and room_cat != "LAB":
                penalty += 5
        
        # 동일 요일 내 다른 세션과 시간 겹침 검사
        for j in range(i+1, n):
            b = schedule[j]
            if a['day'] != b['day']:
                continue
            # 같은 강의실 충돌 검사
            if a['assigned_classroom'] == b['assigned_classroom']:
                if not (a['end_hour'] <= b['start_hour'] or b['end_hour'] <= a['start_hour']):
                    penalty += 3
            # 같은 강사 충돌 검사
            if a['lecturer'] == b['lecturer']:
                if not (a['end_hour'] <= b['start_hour'] or b['end_hour'] <= a['start_hour']):
                    penalty += 3
                    
    return penalty


# -------------------------------
# 5. 랜덤 서치를 통한 최적 스케줄 탐색
# -------------------------------
def random_search(df_sessions, classroom, iterations=10000):
    best_schedule = None
    best_score = float('inf')
    for it in range(iterations):
        sched = generate_random_schedule(df_sessions, classroom)
        score = score_schedule(sched, df_sessions, classroom)
        if score < best_score:
            best_score = score
            best_schedule = sched
        if it % 100 == 0:
            print(f"Iteration {it}: Best score so far = {best_score}")
    return best_schedule, best_score


# -------------------------------
# 실행 예시
# -------------------------------
# offer와 classroom 데이터는 이미 전처리되어 있다고 가정합니다.
best_schedule, best_score = random_search(df_sessions, classroom, iterations=10000)

print("\n최종 스케줄 점수:", best_score)
print("최적 스케줄 할당:")
for assignment in best_schedule:
    print(assignment)


총 세션 수 (분할 후): 870
Iteration 0: Best score so far = 10708
Iteration 100: Best score so far = 10129
Iteration 200: Best score so far = 10129
Iteration 300: Best score so far = 10129
Iteration 400: Best score so far = 10129
Iteration 500: Best score so far = 9988
Iteration 600: Best score so far = 9958
Iteration 700: Best score so far = 9958
Iteration 800: Best score so far = 9958
Iteration 900: Best score so far = 9958
Iteration 1000: Best score so far = 9958
Iteration 1100: Best score so far = 9958
Iteration 1200: Best score so far = 9958
Iteration 1300: Best score so far = 9958
Iteration 1400: Best score so far = 9958
Iteration 1500: Best score so far = 9958
Iteration 1600: Best score so far = 9946
Iteration 1700: Best score so far = 9946
Iteration 1800: Best score so far = 9946
Iteration 1900: Best score so far = 9946
Iteration 2000: Best score so far = 9946
Iteration 2100: Best score so far = 9946
Iteration 2200: Best score so far = 9925
Iteration 2300: Best score so far = 9925
Iter

In [ ]:
import pandas as pd
import numpy as np
import random
import math

# -------------------------------
# 전처리된 offer 데이터 (df_sessions)와 classroom 데이터가 이미 로드되어 있다고 가정
# df_sessions: offer 데이터 전처리 후, 세션 분할 및 인덱스 재설정된 DataFrame
# classroom: 전처리된 classroom DataFrame
# days: 월~금, 시간 슬롯: 08시 ~ 21시 (1시간 단위)
# -------------------------------
days = ["Mon", "Tue", "Wed", "Thu", "Fri"]

def get_valid_start_hours(duration_hours):
    """
    주어진 세션의 지속시간(duration_hours)에 대해,
    시작시간(start_hour) + duration_hours가 21시 이하가 되는 유효한 시작시간 리스트 반환
    """
    return [h for h in range(8, 22) if h + duration_hours <= 21]

# -------------------------------
# 기존 랜덤 서치에서 사용한 score_schedule 함수 (패널티 기반 평가)
# -------------------------------
def score_schedule(schedule, df_sessions, classroom):
    """
    스케줄의 제약 위반에 대해 페널티를 부여합니다.
    
    제약 조건:
      - 강의실이나 시작시간 미할당: +5
      - 시간대 범위 (08~21시) 벗어나면: +3
      - 강의실 정보 누락: +3
      - 강의실 용량 부족: +3
      - Lab 강의인데 Lab 강의실이 아니면: +3
      - 동일 요일 내 동일 강의실 또는 동일 강사의 시간 충돌: 각 충돌 당 +5
    """
    penalty = 0
    n = len(schedule)
    for i in range(n):
        a = schedule[i]
        if a['assigned_classroom'] is None or a['start_hour'] is None:
            penalty += 5
            continue
        if a['start_hour'] < 8 or a['end_hour'] > 21:
            penalty += 3
        cls = classroom[classroom['Resource Code'] == a['assigned_classroom']]
        if cls.empty:
            penalty += 3
        else:
            room_capacity = cls.iloc[0]['Capacity']
            required_capacity = df_sessions.iloc[a['session_index']]['Capacity']
            if room_capacity < required_capacity:
                penalty += 3
            room_cat = cls.iloc[0]['Category'].upper()
            if a['category'].upper() == "LAB" and room_cat != "LAB":
                penalty += 3
        for j in range(i+1, n):
            b = schedule[j]
            if a['day'] != b['day']:
                continue
            if a['assigned_classroom'] == b['assigned_classroom']:
                if not (a['end_hour'] <= b['start_hour'] or b['end_hour'] <= a['start_hour']):
                    penalty += 5
            if a['lecturer'] == b['lecturer']:
                if not (a['end_hour'] <= b['start_hour'] or b['end_hour'] <= a['start_hour']):
                    penalty += 5
    return penalty

# -------------------------------
# GA에서 사용할 개별 유전자(세션 배정) 생성 함수
# -------------------------------
def create_gene(i):
    """
    df_sessions의 i번째 세션에 대해, 강의실, 요일, 시작시간을 무작위로 할당한 유전자 생성
    """
    session = df_sessions.iloc[i]
    duration_hours = math.ceil(session['Min Per Session'] / 60)
    
    # 강의실 후보: 용량 조건
    valid_rooms = classroom[classroom['Capacity'] >= session['Capacity']]
    session_cat = session['Category'].upper()
    if session_cat == "LAB":
        valid_rooms = valid_rooms[valid_rooms['Category'].str.upper() == "LAB"]
    else:
        valid_rooms = valid_rooms[valid_rooms['Category'].str.upper().isin(["GENERAL", "LECTURE"])]
    if valid_rooms.empty:
        assigned_room = None
    else:
        assigned_room = random.choice(valid_rooms['Resource Code'].tolist())
    
    assigned_day = random.choice(days)
    valid_hours = get_valid_start_hours(duration_hours)
    if not valid_hours:
        assigned_hour = None
    else:
        assigned_hour = random.choice(valid_hours)
    
    gene = {
        'session_index': i,
        'course': session['CourseCode'],
        'lecturer': session['Lecturer'],
        'category': session['Category'],
        'required_duration': duration_hours,
        'assigned_classroom': assigned_room,
        'day': assigned_day,
        'start_hour': assigned_hour,
        'end_hour': assigned_hour + duration_hours if assigned_hour is not None else None
    }
    return gene

def create_individual():
    """
    모든 세션에 대해 유전자를 생성하여 개체(시간표)를 반환
    """
    return [create_gene(i) for i in range(len(df_sessions))]

# -------------------------------
# 개체 평가 함수: 페널티 기반 평가를 이용해 fitness 계산
# fitness = 1/(1 + penalty) (패널티가 낮을수록 fitness가 높음)
# -------------------------------
def evaluate_individual(individual):
    penalty = score_schedule(individual, df_sessions, classroom)
    fitness = 1 / (1 + penalty)
    return fitness

# -------------------------------
# Tournament selection
# -------------------------------
def tournament_selection(population, fitnesses, tournament_size):
    selected = []
    for _ in range(len(population)):
        tournament_indices = random.sample(range(len(population)), tournament_size)
        best = None
        best_fit = -1
        for idx in tournament_indices:
            if fitnesses[idx] > best_fit:
                best_fit = fitnesses[idx]
                best = population[idx]
        selected.append(best)
    return selected

# -------------------------------
# One-point crossover
# -------------------------------
def crossover(parent1, parent2):
    size = len(parent1)
    if size < 2:
        return parent1, parent2
    point = random.randint(1, size - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

# -------------------------------
# Mutation: 각 유전자에 대해 일정 확률로 재할당
# -------------------------------
def mutate(individual, mutation_rate):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = create_gene(i)
    return individual

# -------------------------------
# GA 파라미터 설정
# -------------------------------
POP_SIZE = 50
NUM_GENERATIONS = 2000
TOURNAMENT_SIZE = 3
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.1

# -------------------------------
# GA 메인 루프
# -------------------------------
def genetic_algorithm():
    population = [create_individual() for _ in range(POP_SIZE)]
    best_individual = None
    best_fitness = -1
    for gen in range(NUM_GENERATIONS):
        fitnesses = [evaluate_individual(ind) for ind in population]
        # 현재 세대 최고 개체 갱신
        for ind, fit in zip(population, fitnesses):
            if fit > best_fitness:
                best_fitness = fit
                best_individual = ind
        # 선택
        selected = tournament_selection(population, fitnesses, TOURNAMENT_SIZE)
        # 다음 세대 생성: 교차 및 돌연변이 적용
        next_population = []
        while len(next_population) < POP_SIZE:
            p1 = random.choice(selected)
            p2 = random.choice(selected)
            if random.random() < CROSSOVER_RATE:
                child1, child2 = crossover(p1, p2)
            else:
                child1, child2 = p1.copy(), p2.copy()
            child1 = mutate(child1, MUTATION_RATE)
            child2 = mutate(child2, MUTATION_RATE)
            next_population.append(child1)
            if len(next_population) < POP_SIZE:
                next_population.append(child2)
        population = next_population
        if gen % 10 == 0:
            current_penalty = score_schedule(best_individual, df_sessions, classroom)
            print(f"Generation {gen}: Best fitness = {best_fitness:.4f}, Penalty = {current_penalty}")
    return best_individual, best_fitness

# -------------------------------
# GA 실행
# -------------------------------
best_individual, best_fit = genetic_algorithm()
print("\n최적 개체 (Genetic Algorithm 결과):")
print("Best fitness:", best_fit)
print("Best individual penalty:", 1/best_fit - 1)

# -------------------------------
# 시간표 출력 함수 (이전의 print_timetable 함수 활용)
# -------------------------------
def print_timetable(schedule):
    days_order = ["Mon", "Tue", "Wed", "Thu", "Fri"]
    for day in days_order:
        print(f"----- {day} -----")
        day_assignments = [a for a in schedule if a['day'] == day]
        day_assignments.sort(key=lambda x: x['start_hour'] if x['start_hour'] is not None else 99)
        if not day_assignments:
            print("No sessions scheduled.")
        else:
            for a in day_assignments:
                print(f"{a['start_hour']:02d}:00 ~ {a['end_hour']:02d}:00 | Course: {a['course']} ({a['category']}) | Room: {a['assigned_classroom']} | Lecturer: {a['lecturer']}")
        print()

print("\n최적 시간표 (Genetic Algorithm 결과):")
print_timetable(best_individual)


Generation 0: Best fitness = 0.0001, Penalty = 15650
Generation 10: Best fitness = 0.0001, Penalty = 14480
Generation 20: Best fitness = 0.0001, Penalty = 14110
Generation 30: Best fitness = 0.0001, Penalty = 14095
Generation 40: Best fitness = 0.0001, Penalty = 13890
Generation 50: Best fitness = 0.0001, Penalty = 13890
Generation 60: Best fitness = 0.0001, Penalty = 13890
Generation 70: Best fitness = 0.0001, Penalty = 13715
Generation 80: Best fitness = 0.0001, Penalty = 13715
Generation 90: Best fitness = 0.0001, Penalty = 13715
Generation 100: Best fitness = 0.0001, Penalty = 13715
Generation 110: Best fitness = 0.0001, Penalty = 13715
Generation 120: Best fitness = 0.0001, Penalty = 13715
Generation 130: Best fitness = 0.0001, Penalty = 13715
Generation 140: Best fitness = 0.0001, Penalty = 13715
Generation 150: Best fitness = 0.0001, Penalty = 13715
Generation 160: Best fitness = 0.0001, Penalty = 13715
Generation 170: Best fitness = 0.0001, Penalty = 13715
Generation 180: Best 

In [9]:
# best_schedule은 이미 랜덤 서치를 통해 구한 최적의 시간표 리스트라고 가정합니다.
# 각 항목은 { 'session_index', 'course', 'lecturer', 'category', 'required_duration',
#               'assigned_classroom', 'day', 'start_hour', 'end_hour' } 의 딕셔너리입니다.

def print_timetable(schedule):
    # 요일별로 그룹화
    days_order = ["Mon", "Tue", "Wed", "Thu", "Fri"]
    for day in days_order:
        print(f"----- {day} -----")
        # 해당 요일의 배정된 세션 필터링 후 시작시간 순으로 정렬
        day_assignments = [a for a in schedule if a['day'] == day]
        day_assignments.sort(key=lambda x: x['start_hour'] if x['start_hour'] is not None else 99)
        
        if not day_assignments:
            print("No sessions scheduled.")
        else:
            for a in day_assignments:
                # 강의실, 강의, 강사, 시간대, 세션 정보 출력
                print(f"{a['start_hour']:02d}:00 ~ {a['end_hour']:02d}:00 | Course: {a['course']} ({a['category']}) | Room: {a['assigned_classroom']} | Lecturer: {a['lecturer']}")
        print()

# 최적의 시간표 출력 (랜덤 서치 이후)
print_timetable(best_schedule)


----- Mon -----
08:00 ~ 09:00 | Course: BAB1053 (LECTURE) | Room: C314 | Lecturer: ASSOC. PROF. DR. ERIC CHAN WEI CHIANG
08:00 ~ 09:00 | Course: BBB1023 (TUTORIAL) | Room: MPH - GBS | Lecturer: DR. WONG HOONG SANG
08:00 ~ 11:00 | Course: BHH1113 (LECTURE) | Room: MPH - GBS | Lecturer: DR. MARWA REFAAT MAHMOUD AHMED
08:00 ~ 11:00 | Course: BIC1144T (LECTURE) | Room: MPH - GBS | Lecturer: TBA5
08:00 ~ 09:00 | Course: BP263 (LECTURE) | Room: MPH - GBS | Lecturer: ASST. PROF. DR. SASIKALA CHINNAPPAN
08:00 ~ 09:00 | Course: BP373 (TUTORIAL) | Room: MPH - GBS | Lecturer: SABREEN YOUSIF MOHAMMED ALHASSAN NASR
08:00 ~ 09:00 | Course: BPC3142 (TUTORIAL) | Room: C314 | Lecturer: ASST. PROF. DR. CHEW YIK LING
08:00 ~ 10:00 | Course: CET09 (LECTURE) | Room: MPH - GBS | Lecturer: JONATHAN A/L ABISHEGAM
08:00 ~ 10:00 | Course: DEX1034 (ART) | Room: C314 | Lecturer: TS. AMAR RIDZUAN ABD HAMID
08:00 ~ 11:00 | Course: DFS121 (LECTURE) | Room: ANALYTICAL & SENSING, INSTRUMENTATIONS & CONTROL SYSTEM LABO